In [1]:
#This is my project for movie sentiment analysis. 
#i will be using NAIVE-BAYES THEOREM(A simplified version of the Bayes Theorem, known as the Naive Bayes Classification) to make the prediction
#the steps will be 1)text pre procesing , 1)vectorizing , 3)training the mode


In [2]:
import numpy as np 
import pandas as pd
import math

In [3]:
df=pd.read_csv('IMDB Dataset.csv')

df['sentiment'].replace({'positive':1,'negative':0},inplace=True)
df.head()
from sklearn.model_selection import train_test_split
X_test = df.tail(1000).copy()
Y_test=X_test.iloc[:,-1].values
Y_result=np.zeros(1000)
ind=0

In [4]:
#for text processingn i will follow these steps
#remove html tag
#remove special char
#covert all to lower
#remove stop word
#stemming


In [5]:
df=df.head(49000)
df.shape

(49000, 2)

In [6]:
import re

In [7]:
def clean_html(text):
    clean=re.compile('<.*?>')
    return re.sub(clean,'',text)
    

In [8]:
def convert_low(text):
    return text.lower()

In [9]:
def remove_special(text):
    x=''
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [10]:
import nltk
from nltk.corpus import stopwords

In [11]:
def remove_stopper(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y

In [12]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [13]:
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [14]:
def join_back(list_input):
    return " ".join(list_input)

In [15]:
df['review']=df['review'].apply(clean_html)
df['review']=df['review'].apply(convert_low)
df['review']=df['review'].apply(remove_special)
df['review']=df['review'].apply(remove_stopper)
df['review']=df['review'].apply(stem_words)
df['review']=df['review'].apply(join_back)

In [16]:
df

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1
...,...,...
48995,recent saw movi first time enjoy much went rig...,1
48996,film seen anybodi interest effect suffer eat d...,1
48997,person disdain jerri springer show howev found...,1
48998,georg lopez never caught interest stand comedi...,1


In [17]:
#X=df.iloc[:,0:1].values
#print(X)
#X.shape


In [18]:
unique_words = df['review'].str.split().explode().unique().tolist()

word_counts = df['review'].str.split().explode().value_counts()
top_10000_words = word_counts.head(10000).index.tolist()


#print(unique_words)
conditional_prob = {None: {0: None, 1: None}}


positive_reviews = df[df['sentiment'] == 1]

total_positive_words = positive_reviews['review'].str.split().apply(len).sum()
#print(f'Total number of words in positive reviews: {total_positive_words}')

top_words_maskp = positive_reviews['review'].str.split().explode().isin(top_10000_words)
#print(top_words_maskp.value_counts())
top_10_p=top_words_maskp.sum();

pu = positive_reviews['review'].str.split().explode().unique().tolist()
pu = [word for word in pu if word in top_10000_words]
cla_p=len(pu)

negative_reviews = df[df['sentiment'] == 0]
total_negative_words = negative_reviews['review'].str.split().apply(len).sum()
#print(f'Total number of words in negative reviews: {total_negative_words}')

top_words_maskn = negative_reviews['review'].str.split().explode().isin(top_10000_words)
#print(top_words_maskn.value_counts())
top_10_n=top_words_maskn.sum();

nu = negative_reviews['review'].str.split().explode().unique().tolist()
nu = [word for word in nu if word in top_10000_words]
cla_n=len(nu)



for specific_word in top_10000_words:
    
    conditional_prob[specific_word] = {0: None, 1: None}
    
    word_count = positive_reviews['review'].str.count(specific_word).sum()
    conditional_prob[specific_word][1]=(word_count+1)/(top_10_p+cla_p)
    
    word_count = negative_reviews['review'].str.count(specific_word).sum()
    conditional_prob[specific_word][0]=(word_count+1)/(top_10_n+cla_n)



In [19]:
#print(conditional_prob)

In [20]:

def bayes(s):
    
    s=clean_html(s)
    s=convert_low(s)
    s=remove_special(s)
    s=remove_stopper(s)
    s=stem_words(s)
    #print(s)

    total_count = df.shape[0]  
    pos_count = df['sentiment'].sum()  

    prob_pos = pos_count / total_count
    prob_neg = 1 - prob_pos  

    pos=math.log(prob_pos)

    neg=math.log(prob_neg)


    for i in s:
        if i in conditional_prob:
            pos=pos+math.log(conditional_prob[i][1])
            neg=neg+math.log(conditional_prob[i][0])
    global ind
    if(pos>=neg):
        Y_result[ind]=1
    ind=ind+1
    return

In [21]:
X_test['review'].apply(bayes)
global ind
ind=0

In [22]:
from sklearn.metrics import accuracy_score
print("Accuracy",accuracy_score(Y_test,Y_result))

Accuracy 0.83
